# Gröbner Basis Complexity Estimate For Arion & ArionHash

In this SageMath notebook we estimate the complexity of Gröbner basis attacks on Arion & ArionHash.

In [1]:
def log2(x):
    return log(x) / log(2)

def stirling_approximation(n, k):
    """
    Computes the logarithm in base 2 of Stirling's binomial coefficient approximation.
    """
    approx_1 = n - k
    approx_1 *= log2(n / (n - k))
    approx_1 += k * log(n / k)
    approx_2 = log2(n)
    approx_2 -= log2(pi)
    approx_2 -= log2(k)
    approx_2 -= log2(n - k)
    approx_2 /= 2
    return approx_1 + approx_2

The cost of probabilistic polynomial system solving is estimated via
\begin{equation*}
    \mathcal{O} \left( n \cdot d^\omega + d \cdot \log \left( q \right) \cdot \log \left( d \right) \cdot \log \big( \log \left( d \right) \big) + d \cdot \log \left( d \right)^2 \cdot \log \big( \log \left( d \right) \big) \right),
\end{equation*}
and with deterministic methods via
\begin{equation*}
    \mathcal{O} \left( \sqrt{n} \cdot d^{2 + \frac{n - 1}{n}} + d \cdot \log \left( q \right) \cdot \log \left( d \right) \cdot \log \big( \log \left( d \right) \big) + d \cdot \log \left( d \right)^2 \cdot \log \big( \log \left( d \right) \big) \right),
\end{equation*}
where $q$ is the size of the finite field, $n$ the number of variables, $d$ the vector space dimension of the quotient space and $\omega \geq 2$ a linear algebra constant.

In [2]:
def complexity_deterministic_solving(n, d, q):
    compl = sqrt(n) * d**(2 + (n - 1) / n)
    compl += d * log(q) * log(d) * log(log(d))
    compl += d * log(d)**2 * log(log(d))
    compl = log2(compl)
    return float(compl)

def complexity_probabilistic_solving(n, d, q, omega=2):
    compl = n * d**omega
    if d < q:
        compl += d * log(q) * log(d) * log(log(d))
        compl += d * log(d)**2 * log(log(d))
    else:
        compl += q * log(d) * log(q) * log(log(q))
        compl += q * log(q)**2 * log(log(q))
    compl = log2(compl)
    return float(compl)

The cost of Gröbner basis computations is estimated via
\begin{equation*}
    \mathcal{O} \left( \binom{n + d}{n}^\omega \right),
\end{equation*}
where $n$ is the number of variables, $d$ is the solving degree and $\omega \geq 2$ a linear algebra constant.

In [3]:
def complexity_gb_computation(n, d, omega=2, stirling=False):
    if stirling:
        return float(omega * stirling_approximation(n + d, d))
    else:
        return float(omega * log2(binomial(n + d, d)))

In [4]:
def arion_macaulay_bound(n, r, d_1, d_2):
    out = d_2 + 1
    for i in range(1, n):
        out += 2**(n - i) * (d_1 + 1) - d_1
    out *= r
    out += 1
    out -= r * (n + 1)
    return out

In [5]:
def arion_GTDS_maximal_degree(n, d_1, d_2):
    return 2**(n - 1) * (d_1 + d_2) - d_1

### Table Legend

 - GB_MB ... complexity of Gröbner basis computation if Macaulay bound is used as solving degree
 - GB_min ... complexity of Gröbner basis computation if highest degree in polynomial system is used as solving degree
 - Solving_det ... complexity of polynomial system solving with deterministic algorithm
 - Solving_prob ... complexity of polynomial system solving with probabilitstic algorithm

All complexities are given in bits.

## Arion

We hypothesize that the quotient space dimension of Arion is

$$ \dim_{\mathbb{F}_p} \left( \mathcal{F}_\textsf{Arion} \right) (n, r, d_1, d_2) = \left( d_2 \cdot \left( d_1 + 2 \right)^{n - 1} \right)^r, \qquad n \geq 1. $$

In [6]:
def arion_quotient_space_dimension(n, r, d_1, d_2):
    return (d_2 * (d_1 + 2)**(n - 1))**r

In [7]:
rounds = 12
branches = 8
sizes = [60, 120, 250] # 2^N

omega = 2
security_bound = 128

print("Arion", "\n")
print("Minimum number of rounds for", security_bound, "bit security for system solving.", "\n")
print("omega:", omega, "\n")

print("N", "\t",
      "n", "\t", 
      "r", "\t",
      "d_1", "\t", 
      "d_2", "\t",  
      "GB_min", "\t", 
      "GB_MB", "\t",
      "Solving_det", "\t", 
      "Solving_prob")
for N in sizes:
    q = 2**N
    for n in range(3, branches + 1):
        d_1 = 3
        d_2 = 121
        r = 1
        n_vars = (n + 1) * r
        sd_mb = arion_macaulay_bound(n, r, d_1, d_2)              # solvind degree Macaulay bound
        sd_min = max([d_2, arion_GTDS_maximal_degree(n, d_1, 1)]) # minimal possible solving degree
        d = arion_quotient_space_dimension(n, r, d_1, d_2)
        compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
        compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
        while compl_solv_prob < security_bound:
            r += 1
            n_vars = (n + 1) * r
            d = arion_quotient_space_dimension(n, r, d_1, d_2)
            compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
            compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
        compl_gb_mb = floor(complexity_gb_computation(n_vars, sd_mb, omega, True))
        compl_gb_min = floor(complexity_gb_computation(n_vars, sd_min, omega, True))
        print(N, "\t",
              n, "\t", 
              r, "\t", 
              d_1, "\t", 
              d_2, "\t", 
              compl_gb_min, "\t\t",
              compl_gb_mb, "\t",
              compl_solv_det, "\t\t",
              compl_solv_prob)
                
        d_1 = 5
        d_2 = 121
        r = 1
        n_vars = (n + 1) * r
        sd_mb = arion_macaulay_bound(n, r, d_1, d_2)              # solvind degree Macaulay bound
        sd_min = max([d_2, arion_GTDS_maximal_degree(n, d_1, 1)]) # minimal possible solving degree
        d = arion_quotient_space_dimension(n, r, d_1, d_2)
        compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
        compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
        while compl_solv_prob < security_bound:
            r += 1
            n_vars = (n + 1) * r
            d = arion_quotient_space_dimension(n, r, d_1, d_2)
            compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
            compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
        compl_gb_mb = floor(complexity_gb_computation(n_vars, sd_mb, omega, True))
        compl_gb_min = floor(complexity_gb_computation(n_vars, sd_min, omega, True))
        print(N, "\t",
              n, "\t", 
              r, "\t", 
              d_1, "\t", 
              d_2, "\t", 
              compl_gb_min, "\t\t",
              compl_gb_mb, "\t",
              compl_solv_det, "\t\t",
              compl_solv_prob)
        print("")

Arion 

Minimum number of rounds for 128 bit security for system solving. 

omega: 2 

N 	 n 	 r 	 d_1 	 d_2 	 GB_min 	 GB_MB 	 Solving_det 	 Solving_prob
60 	 3 	 6 	 3 	 121 	 162 		 170 	 207 		 143
60 	 3 	 5 	 5 	 121 	 143 		 153 	 187 		 129

60 	 4 	 5 	 3 	 121 	 166 		 186 	 207 		 143
60 	 4 	 5 	 5 	 121 	 166 		 195 	 229 		 158

60 	 5 	 4 	 3 	 121 	 162 		 201 	 194 		 134
60 	 5 	 4 	 5 	 121 	 162 		 215 	 217 		 149

60 	 6 	 4 	 3 	 121 	 181 		 257 	 222 		 153
60 	 6 	 3 	 5 	 121 	 172 		 225 	 187 		 130

60 	 7 	 3 	 3 	 121 	 209 		 266 	 187 		 129
60 	 7 	 3 	 5 	 121 	 235 		 289 	 213 		 147

60 	 8 	 3 	 3 	 121 	 279 		 338 	 208 		 143
60 	 8 	 3 	 5 	 121 	 309 		 366 	 238 		 164

120 	 3 	 6 	 3 	 121 	 162 		 170 	 207 		 143
120 	 3 	 5 	 5 	 121 	 143 		 153 	 187 		 129

120 	 4 	 5 	 3 	 121 	 166 		 186 	 207 		 143
120 	 4 	 5 	 5 	 121 	 166 		 195 	 229 		 158

120 	 5 	 4 	 3 	 121 	 162 		 201 	 194 		 134
120 	 5 	 4 	 5 	 121 	 162 		 21

### Aggressive Arion

In [8]:
rounds = 12
branches = 8
sizes = [60, 120, 250] # 2^N

omega = 2
security_bound = 128

print("Aggressive Arion", "\n")
print("Minimum number of rounds for", security_bound, "bit security for system solving.", "\n")
print("omega:", omega, "\n")

print("N", "\t",
      "n", "\t", 
      "r", "\t",
      "d_1", "\t", 
      "d_2", "\t",  
      "GB_min", "\t",
      "GB_MB", "\t", 
      "Solving_det", "\t", 
      "Solving_prob")
for N in sizes:
    q = 2**N
    for n in range(3, branches + 1):
        d_1 = 3
        m = 1
        d_2 = 2**m + 1
        r = 4
        n_vars = (n + 1) * r
        sd_mb = arion_macaulay_bound(n, r, d_1, d_2)              # solvind degree Macaulay bound
        sd_min = max([d_2, arion_GTDS_maximal_degree(n, d_1, 1)]) # minimal possible solving degree
        d = arion_quotient_space_dimension(n, r, d_1, d_2)
        compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
        compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
        while compl_solv_det < security_bound or compl_solv_prob < security_bound:
            m += 1
            d_2 = 2**m + 1
            n_vars = (n + 1) * r
            d = arion_quotient_space_dimension(n, r, d_1, d_2)
            compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
            compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
        compl_gb_mb = floor(complexity_gb_computation((n + 1) * r, sd_mb, omega, True))
        compl_gb_min = floor(complexity_gb_computation((n + 1) * r, sd_min, omega, True))
        print(N, "\t",
              n, "\t", 
              r, "\t", 
              d_1, "\t", 
              d_2, "\t", 
              compl_gb_min, "\t\t",
              compl_gb_mb, "\t",
              compl_solv_det, "\t\t",
              compl_solv_prob)
                
        d_1 = 5
        m = 1
        d_2 = 2**m + 1
        r = 4
        n_vars = (n + 1) * r
        sd_mb = arion_macaulay_bound(n, r, d_1, d_2)              # solvind degree Macaulay bound
        sd_min = max([d_2, arion_GTDS_maximal_degree(n, d_1, 1)]) # minimal possible solving degree
        d = arion_quotient_space_dimension(n, r, d_1, d_2)
        compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
        compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
        while compl_solv_det < security_bound or compl_solv_prob < security_bound:
            m += 1
            d_2 = 2**m + 1
            n_vars = (n + 1) * r
            d = arion_quotient_space_dimension(n, r, d_1, d_2)
            compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
            compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
        compl_gb_mb = floor(complexity_gb_computation((n + 1) * r, sd_mb, omega, True))
        compl_gb_min = floor(complexity_gb_computation((n + 1) * r, sd_min, omega, True))
        print(N, "\t",
              n, "\t", 
              r, "\t", 
              d_1, "\t", 
              d_2, "\t", 
              compl_gb_min, "\t\t",
              compl_gb_mb, "\t",
              compl_solv_det, "\t\t",
              compl_solv_prob)
        print("")

Aggressive Arion 

Minimum number of rounds for 128 bit security for system solving. 

omega: 2 

N 	 n 	 r 	 d_1 	 d_2 	 GB_min 	 GB_MB 	 Solving_det 	 Solving_prob
60 	 3 	 4 	 3 	 2049 	 43 		 102 	 185 		 129
60 	 3 	 4 	 5 	 1025 	 54 		 117 	 185 		 128

60 	 4 	 4 	 3 	 513 	 76 		 166 	 190 		 132
60 	 4 	 4 	 5 	 257 	 93 		 187 	 196 		 135

60 	 5 	 4 	 3 	 129 	 122 		 243 	 195 		 134
60 	 5 	 4 	 5 	 33 	 145 		 270 	 194 		 134

60 	 6 	 4 	 3 	 17 	 181 		 333 	 188 		 130
60 	 6 	 4 	 5 	 3 	 210 		 365 	 187 		 129

60 	 7 	 4 	 3 	 3 	 255 		 436 	 186 		 129
60 	 7 	 4 	 5 	 3 	 290 		 473 	 221 		 152

60 	 8 	 4 	 3 	 3 	 345 		 554 	 214 		 147
60 	 8 	 4 	 5 	 3 	 385 		 595 	 255 		 175

120 	 3 	 4 	 3 	 2049 	 43 		 102 	 185 		 129
120 	 3 	 4 	 5 	 1025 	 54 		 117 	 185 		 128

120 	 4 	 4 	 3 	 513 	 76 		 166 	 190 		 132
120 	 4 	 4 	 5 	 257 	 93 		 187 	 196 		 135

120 	 5 	 4 	 3 	 129 	 122 		 243 	 195 		 134
120 	 5 	 4 	 5 	 33 	 145 		 270 	 19

## ArionHash

We hypothesize that the quotient space dimension of ArionHash is


$$ \dim_{\mathbb{F}_p} \left( \mathcal{F}_\textsf{ArionHash} \right) (n, r, d_1, d_2) = \Big( 2^{n - 1} \cdot d_2 \cdot \left( d_1 + 1 \right) - d_1 \cdot d_2 \Big)^r, \qquad n \geq 1. $$

In [9]:
def arion_hash_quotient_space_dimension(n, r, d_1, d_2):
    return (2**(n - 1) * d_2 * (d_1 + 1) - d_1 * d_2)**r

In [10]:
rounds = 12
branches = 8
sizes = [60, 120, 250] # 2^N

omega = 2
security_bound = 128

print("ArionHash", "\n")
print("Minimum number of rounds for", security_bound, "bit security for system solving.", "\n")
print("omega:", omega, "\n")

print("N", "\t",
      "n", "\t", 
      "r", "\t",
      "d_1", "\t", 
      "d_2", "\t",  
      "GB_min", "\t",
      "GB_MB", "\t", 
      "Solving_det", "\t", 
      "Solving_prob")
for N in sizes:
    q = 2**N
    for n in range(3, branches + 1):
        d_1 = 3
        d_2 = 121
        r = 1
        n_vars = (n + 1) * r
        sd_mb = arion_macaulay_bound(n, r, d_1, d_2)              # solvind degree Macaulay bound
        sd_min = max([d_2, arion_GTDS_maximal_degree(n, d_1, 1)]) # minimal possible solving degree
        d = arion_hash_quotient_space_dimension(n, r, d_1, d_2)
        compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
        compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
        while compl_solv_prob < security_bound:
            r += 1
            n_vars = (n + 1) * r
            d = arion_hash_quotient_space_dimension(n, r, d_1, d_2)
            compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
            compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
        compl_gb_mb = floor(complexity_gb_computation((n + 1) * r, sd_mb, omega, True))
        compl_gb_min = floor(complexity_gb_computation((n + 1) * r, sd_min, omega, True))
        print(N, "\t",
              n, "\t", 
              r, "\t", 
              d_1, "\t", 
              d_2, "\t", 
              compl_gb_min, "\t\t",
              compl_gb_mb, "\t",
              compl_solv_det, "\t\t",
              compl_solv_prob)
                
        d_1 = 5
        d_2 = 121
        r = 1
        n_vars = (n + 1) * r
        sd_mb = arion_macaulay_bound(n, r, d_1, d_2)              # solvind degree Macaulay bound
        sd_min = max([d_2, arion_GTDS_maximal_degree(n, d_1, 1)]) # minimal possible solving degree
        d = arion_hash_quotient_space_dimension(n, r, d_1, d_2)
        compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
        compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
        while compl_solv_prob < security_bound:
            r += 1
            n_vars = (n + 1) * r
            d = arion_hash_quotient_space_dimension(n, r, d_1, d_2)
            compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
            compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
        compl_gb_mb = floor(complexity_gb_computation((n + 1) * r, sd_mb, omega, True))
        compl_gb_min = floor(complexity_gb_computation((n + 1) * r, sd_min, omega, True))
        print(N, "\t",
              n, "\t", 
              r, "\t", 
              d_1, "\t", 
              d_2, "\t", 
              compl_gb_min, "\t\t",
              compl_gb_mb, "\t",
              compl_solv_det, "\t\t",
              compl_solv_prob)
        print("")

ArionHash 

Minimum number of rounds for 128 bit security for system solving. 

omega: 2 

N 	 n 	 r 	 d_1 	 d_2 	 GB_min 	 GB_MB 	 Solving_det 	 Solving_prob
60 	 3 	 6 	 3 	 121 	 162 		 170 	 190 		 132
60 	 3 	 6 	 5 	 121 	 162 		 173 	 200 		 138

60 	 4 	 6 	 3 	 121 	 187 		 210 	 212 		 146
60 	 4 	 5 	 5 	 121 	 166 		 195 	 185 		 128

60 	 5 	 5 	 3 	 121 	 187 		 235 	 193 		 133
60 	 5 	 5 	 5 	 121 	 187 		 251 	 201 		 139

60 	 6 	 5 	 3 	 121 	 208 		 301 	 208 		 143
60 	 6 	 5 	 5 	 121 	 244 		 328 	 217 		 149

60 	 7 	 5 	 3 	 121 	 297 		 390 	 224 		 154
60 	 7 	 4 	 5 	 121 	 290 		 361 	 186 		 128

60 	 8 	 4 	 3 	 121 	 345 		 423 	 191 		 132
60 	 8 	 4 	 5 	 121 	 385 		 461 	 198 		 137

120 	 3 	 6 	 3 	 121 	 162 		 170 	 190 		 132
120 	 3 	 6 	 5 	 121 	 162 		 173 	 200 		 138

120 	 4 	 6 	 3 	 121 	 187 		 210 	 212 		 146
120 	 4 	 5 	 5 	 121 	 166 		 195 	 185 		 128

120 	 5 	 5 	 3 	 121 	 187 		 235 	 193 		 133
120 	 5 	 5 	 5 	 121 	 187 	

### Aggressive ArionHash

In [11]:
rounds = 12
branches = 8
sizes = [60, 120, 250] # 2^N

omega = 2
security_bound = 128

print("Aggressive ArionHash", "\n")
print("Minimum number of rounds for", security_bound, "bit security for system solving.", "\n")
print("omega:", omega, "\n")

print("N", "\t",
      "n", "\t", 
      "r", "\t",
      "d_1", "\t", 
      "d_2", "\t",  
      "GB_min", "\t",
      "GB_MB", "\t", 
      "Solving_det", "\t", 
      "Solving_prob")
for N in sizes:
    q = 2**N
    for n in range(3, branches + 1):
        d_1 = 3
        m = 1
        d_2 = 2**m + 1
        r = 4
        n_vars = (n + 1) * r
        sd_mb = arion_macaulay_bound(n, r, d_1, d_2)              # solvind degree Macaulay bound
        sd_min = max([d_2, arion_GTDS_maximal_degree(n, d_1, 1)]) # minimal possible solving degree
        d = arion_hash_quotient_space_dimension(n, r, d_1, d_2)
        compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
        compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
        while compl_solv_prob < security_bound:
            r += 1
            n_vars = (n + 1) * r
            d = arion_hash_quotient_space_dimension(n, r, d_1, d_2)
            compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
            compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
        compl_gb_mb = floor(complexity_gb_computation((n + 1) * r, sd_mb, omega, True))
        compl_gb_min = floor(complexity_gb_computation((n + 1) * r, sd_min, omega, True))
        print(N, "\t",
              n, "\t", 
              r, "\t", 
              d_1, "\t", 
              d_2, "\t", 
              compl_gb_min, "\t\t",
              compl_gb_mb, "\t",
              compl_solv_det, "\t\t",
              compl_solv_prob)
                
        d_1 = 5
        m = 1
        d_2 = 2**m + 1
        r = 4
        n_vars = (n + 1) * r
        sd_mb = arion_macaulay_bound(n, r, d_1, d_2)              # solvind degree Macaulay bound
        sd_min = max([d_2, arion_GTDS_maximal_degree(n, d_1, 1)]) # minimal possible solving degree
        d = arion_hash_quotient_space_dimension(n, r, d_1, d_2)
        compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
        compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
        while compl_solv_prob < security_bound:
            r += 1
            n_vars = (n + 1) * r
            d = arion_hash_quotient_space_dimension(n, r, d_1, d_2)
            compl_solv_det = floor(complexity_deterministic_solving(n_vars, d, q))
            compl_solv_prob = floor(complexity_probabilistic_solving(n_vars, d, q, omega))
        compl_gb_mb = floor(complexity_gb_computation((n + 1) * r, sd_mb, omega, True))
        compl_gb_min = floor(complexity_gb_computation((n + 1) * r, sd_min, omega, True))
        print(N, "\t",
              n, "\t", 
              r, "\t", 
              d_1, "\t", 
              d_2, "\t", 
              compl_gb_min, "\t\t",
              compl_gb_mb, "\t",
              compl_solv_det, "\t\t",
              compl_solv_prob)
        print("")

Aggressive ArionHash 

Minimum number of rounds for 128 bit security for system solving. 

omega: 2 

N 	 n 	 r 	 d_1 	 d_2 	 GB_min 	 GB_MB 	 Solving_det 	 Solving_prob
60 	 3 	 12 	 3 	 3 	 68 		 195 	 191 		 132
60 	 3 	 11 	 5 	 3 	 85 		 221 	 193 		 133

60 	 4 	 10 	 3 	 3 	 118 		 304 	 194 		 134
60 	 4 	 9 	 5 	 3 	 142 		 330 	 190 		 131

60 	 5 	 9 	 3 	 3 	 188 		 434 	 204 		 141
60 	 5 	 8 	 5 	 3 	 217 		 452 	 195 		 135

60 	 6 	 8 	 3 	 3 	 275 		 563 	 206 		 142
60 	 6 	 7 	 5 	 3 	 302 		 565 	 193 		 133

60 	 7 	 7 	 3 	 3 	 369 		 679 	 202 		 139
60 	 7 	 7 	 5 	 3 	 428 		 743 	 214 		 147

60 	 8 	 6 	 3 	 3 	 460 		 771 	 192 		 132
60 	 8 	 6 	 5 	 3 	 520 		 833 	 202 		 139

120 	 3 	 12 	 3 	 3 	 68 		 195 	 191 		 132
120 	 3 	 11 	 5 	 3 	 85 		 221 	 193 		 133

120 	 4 	 10 	 3 	 3 	 118 		 304 	 194 		 134
120 	 4 	 9 	 5 	 3 	 142 		 330 	 190 		 131

120 	 5 	 9 	 3 	 3 	 188 		 434 	 204 		 141
120 	 5 	 8 	 5 	 3 	 217 		 452 	 195 		 135

120